In [22]:
from ezc3d import c3d



In [23]:
# file = r"C:\Users\anna\Documents\gitHub\NTU\InterpolationModel\data\c3d\SN268\SN268_0004_grasp_R01.c3d"
file = r"C:\Users\Administrator\Documents\gitHub\NTU\InterpolationModel\data\SN268\SN268_0004_grasp_R01.c3d"

c = c3d(file)

In [24]:
name_of_markers = c["parameters"]["POINT"]["DESCRIPTIONS"]["value"]

In [25]:
name_of_markers

['RUA1',
 'RUA2',
 'RUA3',
 'RUA4',
 'RHLE',
 'RHME',
 'RTEMP',
 'RHEAD',
 'LHEAD',
 'LTEMP',
 'STER',
 'XPRO',
 'C7',
 'T4',
 'T8',
 'T10',
 'LPSIS',
 'RPSIS',
 'LICR',
 'RICR',
 'LASIS',
 'RASIS',
 'LUA1',
 'LUA2',
 'LUA3',
 'LUA4',
 'LHLE',
 'LHME',
 'LFA1',
 'LFA2',
 'LFA3',
 'LRSP',
 'LUSP',
 'LCAP',
 'LHMC1',
 'LHMC2',
 'LHMC3',
 'RFA1',
 'RFA2',
 'RFA3',
 'RRSP',
 'RUSP',
 'RCAP',
 'RHMC1',
 'RHMC2',
 'RHMC3',
 'RACR',
 'LACR',
 'LTHUMB',
 'LMIDDLE',
 'RTHUMB',
 'RMIDDLE',
 'TAB1',
 'TAB2',
 'TAB3',
 'TAB4',
 'CHAIR1',
 'CHAIR2',
 'CHAIR3',
 'CHAIR4',
 'BLOCK',
 'SHELF1',
 'SHELF2',
 'SHELF3',
 'SHELF4']

In [26]:
coordinates_of_markers = c["data"]["points"][0:3,:,:]

In [27]:
markers_grouping = markers_dict = {
    'R_head':[
        'RHEAD',
        'RTEMP'],

    'L_head':[
        'LHEAD',
        'LTEMP'],

    'R_shoulder':[
        'RACR'],

    'L_shoulder':[
        'LACR'],

    'torso':[
        'STER',
        'XPRO',
        'C7',
        'T4',
        'T8',
        'T10'],
    

    'pelvis':[
        'LPSIS',
        'RPSIS',
        'RASIS',
        'LASIS',
        'LICR',
        'RICR'],

    'R_arms':[    
        'RHME',
        'RHLE',
        'RUA1',
        'RUA2',
        'RUA3',
        'RUA4'],
    
    'L_arms':[
        'LHME',
        'LHLE',
        'LUA1',
        'LUA2',
        'LUA3',
        'LUA4'],

    'R_wrist':[
        'RRSP',
        'RUSP',
        'RFA3',
        'RFA2',
        'RFA1'],

    'L_wrist':[
        'LFA1',
        'LFA2',
        'LFA3',
        'LRSP',
        'LUSP'],

    'R_hand':[
        'RCAP',
        'RHMC1',
        'RHMC2',
        'RHMC3'],
    
    'L_hand':[
        'LCAP',
        'LHMC1',
        'LHMC2',
        'LHMC3'],

    'R_fingers':[
        'RTHUMB',
        'RMIDDLE'],
    
    'L_fingers':[
        'LTHUMB',
        'LMIDDLE'],

    'R_thigh':[
        'RFLE',
        'RFME',
        'RTH1',
        'RTH2',
        'RTH3',
        'RTH4'],
    
    'L_thigh':[
        'LFLE',
        'LFME',
        'LTH1',
        'LTH2',
        'LTH3',
        'LTH4'],

    'R_calves':[
        'RSK1',
        'RSK2',
        'RSK3',
        'RSK4',
        'RFAL',
        'RTAM'],
        
    'L_calves':[
        'LSK1',
        'LSK2',
        'LSK3',
        'LSK4',
        'LFAL',
        'LTAM'],

    'R_foot':[
        'RFCC',
        'RFMT5',
        'RFMT2',
        'RFMT1'],

    'L_foot':[
        'LFCC',
        'LFMT5',
        'LFMT2',
        'LFMT1']
    }

In [30]:
type(coordinates_of_markers)

numpy.ndarray

In [29]:
# def reorganize_data(coordinates_of_markers, name_of_markers):
#     sequence = [
#                 "L_foot",
#                 "L_calves",
#                 "L_thigh",

                
    ]

SyntaxError: expected ':' (3775832973.py, line 1)